In [1]:
import keras
import numpy as np
import scipy.io
from pathlib import Path
import os

Using TensorFlow backend.


## Open labels

In [2]:
path = 'C:\\Users\\Rodolfo\\Documents\\algoritmos-git\\custom_cnn\\Data\\Processados\\'

In [3]:
y_train = np.load(Path(path,'y_train.npy'))
y_test = np.load(Path(path, 'y_test.npy'))
y_train = np.expand_dims(y_train, axis=1)
y_test = np.expand_dims(y_test, axis=1)

In [4]:
from keras.utils import np_utils
# one-hot encode the labels
num_classes = len(np.unique(y_train))
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [5]:
y_train

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [6]:
y_test

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

# Open data

In [7]:
path = Path(path, '28_28')
X_train_1 = np.load(Path(path, 'X_train_1.npy'))
X_train_2 = np.load(Path(path, 'X_train_2.npy'))
X_test_1 = np.load(Path(path, 'X_test_1.npy'))
X_test_2 = np.load(Path(path, 'X_test_2.npy'))
print(X_train_1.shape)
print(X_train_2.shape)
print(X_test_1.shape)
print(X_test_2.shape)

(2200, 28, 28)
(2200, 28, 28)
(1000, 28, 28)
(1000, 28, 28)


### Shape desejado: x_train shape: (45000, 32, 32, 3) -> n_obs, n_row, n_col, n_chan

In [8]:
n_obs, n_row, n_col = X_train_1.shape
n_chan=2
X_train = np.zeros((n_obs, n_row, n_col, n_chan))
X_train.shape

(2200, 28, 28, 2)

In [9]:
X_train[:,:,:,0]=X_train_1
X_train[:,:,:,1]=X_train_2
X_train.shape

(2200, 28, 28, 2)

In [10]:
n_obs, n_row, n_col = X_test_1.shape
n_chan=2
X_test = np.zeros((n_obs, n_row, n_col, n_chan))
X_test.shape

(1000, 28, 28, 2)

In [11]:
X_test[:,:,:,0]=X_test_1
X_test[:,:,:,1]=X_test_2
X_test.shape

(1000, 28, 28, 2)

# Shuffle X_train

In [12]:
arr = np.arange(y_train.shape[0])
np.random.shuffle(arr)
X_train = X_train[arr,:,:,:]
y_train = y_train[arr]

### 3. Rescale the Images by Dividing Every Pixel in Every Image by 255

In [13]:
# rescale [0,255] --> [0,1]
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

### 4.  Break Dataset into Training, Testing, and Validation Sets

In [14]:
# break training set into training and validation sets
(X_train, X_valid) = X_train[:1800], X_train[1800:]
(y_train, y_valid) = y_train[:1800], y_train[1800:]

# print shape of training set
print('x_train shape:', X_train.shape)

# print number of training, validation, and test images
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
print(X_valid.shape[0], 'validation samples')

x_train shape: (1800, 28, 28, 2)
1800 train samples
1000 test samples
400 validation samples


### 5. Define the Model Architecture 

In [18]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu', strides=(1,1), input_shape=(28, 28, 2)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu', strides=(2,2)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', strides=(1,1)))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 28, 28, 16)        144       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 7, 7, 32)          2080      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 3, 3, 32)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 3, 64)          8256      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 1, 1, 64)          0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 1, 1, 64)          0         
__________

### 6. Compile the Model 

In [19]:
# compile the model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', 
                  metrics=['accuracy'])

### 7. Train the Model 

In [21]:
from keras.callbacks import ModelCheckpoint   

# train the model
checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose=5, 
                               save_best_only=True)
hist = model.fit(X_train, y_train, batch_size=32, epochs=500,
          validation_data=(X_valid, y_valid), callbacks=[checkpointer], 
          verbose=5, shuffle=True)

Train on 1800 samples, validate on 400 samples
Epoch 1/500

Epoch 00001: val_loss improved from inf to 0.66647, saving model to model.weights.best.hdf5
Epoch 2/500

Epoch 00002: val_loss did not improve from 0.66647
Epoch 3/500

Epoch 00003: val_loss did not improve from 0.66647
Epoch 4/500

Epoch 00004: val_loss did not improve from 0.66647
Epoch 5/500

Epoch 00005: val_loss did not improve from 0.66647
Epoch 6/500

Epoch 00006: val_loss did not improve from 0.66647
Epoch 7/500

Epoch 00007: val_loss did not improve from 0.66647
Epoch 8/500

Epoch 00008: val_loss did not improve from 0.66647
Epoch 9/500

Epoch 00009: val_loss did not improve from 0.66647
Epoch 10/500

Epoch 00010: val_loss did not improve from 0.66647
Epoch 11/500

Epoch 00011: val_loss did not improve from 0.66647
Epoch 12/500

Epoch 00012: val_loss did not improve from 0.66647
Epoch 13/500

Epoch 00013: val_loss did not improve from 0.66647
Epoch 14/500

Epoch 00014: val_loss did not improve from 0.66647
Epoch 15/50


Epoch 00126: val_loss did not improve from 0.66647
Epoch 127/500

Epoch 00127: val_loss did not improve from 0.66647
Epoch 128/500

Epoch 00128: val_loss did not improve from 0.66647
Epoch 129/500

Epoch 00129: val_loss did not improve from 0.66647
Epoch 130/500

Epoch 00130: val_loss did not improve from 0.66647
Epoch 131/500

Epoch 00131: val_loss did not improve from 0.66647
Epoch 132/500

Epoch 00132: val_loss did not improve from 0.66647
Epoch 133/500

Epoch 00133: val_loss did not improve from 0.66647
Epoch 134/500

Epoch 00134: val_loss did not improve from 0.66647
Epoch 135/500

Epoch 00135: val_loss did not improve from 0.66647
Epoch 136/500

Epoch 00136: val_loss did not improve from 0.66647
Epoch 137/500

Epoch 00137: val_loss did not improve from 0.66647
Epoch 138/500

Epoch 00138: val_loss did not improve from 0.66647
Epoch 139/500

Epoch 00139: val_loss did not improve from 0.66647
Epoch 140/500

Epoch 00140: val_loss did not improve from 0.66647
Epoch 141/500

Epoch 001


Epoch 00251: val_loss did not improve from 0.66647
Epoch 252/500

Epoch 00252: val_loss did not improve from 0.66647
Epoch 253/500

Epoch 00253: val_loss did not improve from 0.66647
Epoch 254/500

Epoch 00254: val_loss did not improve from 0.66647
Epoch 255/500

Epoch 00255: val_loss did not improve from 0.66647
Epoch 256/500

Epoch 00256: val_loss did not improve from 0.66647
Epoch 257/500

Epoch 00257: val_loss did not improve from 0.66647
Epoch 258/500

Epoch 00258: val_loss did not improve from 0.66647
Epoch 259/500

Epoch 00259: val_loss did not improve from 0.66647
Epoch 260/500

Epoch 00260: val_loss did not improve from 0.66647
Epoch 261/500

Epoch 00261: val_loss did not improve from 0.66647
Epoch 262/500

Epoch 00262: val_loss did not improve from 0.66647
Epoch 263/500

Epoch 00263: val_loss did not improve from 0.66647
Epoch 264/500

Epoch 00264: val_loss did not improve from 0.66647
Epoch 265/500

Epoch 00265: val_loss did not improve from 0.66647
Epoch 266/500

Epoch 002


Epoch 00376: val_loss did not improve from 0.66647
Epoch 377/500

Epoch 00377: val_loss did not improve from 0.66647
Epoch 378/500

Epoch 00378: val_loss did not improve from 0.66647
Epoch 379/500

Epoch 00379: val_loss did not improve from 0.66647
Epoch 380/500

Epoch 00380: val_loss did not improve from 0.66647
Epoch 381/500

Epoch 00381: val_loss did not improve from 0.66647
Epoch 382/500

Epoch 00382: val_loss did not improve from 0.66647
Epoch 383/500

Epoch 00383: val_loss did not improve from 0.66647
Epoch 384/500

Epoch 00384: val_loss did not improve from 0.66647
Epoch 385/500

Epoch 00385: val_loss did not improve from 0.66647
Epoch 386/500

Epoch 00386: val_loss did not improve from 0.66647
Epoch 387/500

Epoch 00387: val_loss did not improve from 0.66647
Epoch 388/500

Epoch 00388: val_loss did not improve from 0.66647
Epoch 389/500

Epoch 00389: val_loss did not improve from 0.66647
Epoch 390/500

Epoch 00390: val_loss did not improve from 0.66647
Epoch 391/500

Epoch 003

### 8. Load the Model with the Best Validation Accuracy

In [22]:
# load the weights that yielded the best validation accuracy
model.load_weights('model.weights.best.hdf5')

### 9. Calculate Classification Accuracy on Test Set

In [23]:
# evaluate and print test accuracy
score = model.evaluate(X_test, y_test, verbose=0)
print('\n', 'Test accuracy:', score[1])


 Test accuracy: 0.684
